In [5]:
#evansdata
#gather time stamps --camera and wf--
import numpy as np
import pandas as pd
import h5py
from datapipe import find_nearest, getData, rowsToColumns
from pathlib import Path
import pickle
import scipy.io as scio
from sklearn.externals import joblib
from skimage import io
import tensorflow as tf

In [6]:
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
#should respond to audio

im1 = io.imread('./data/whisk.tif')
(length, y, x) = im1.shape
# im2 = io.imread('./data/AV_5_1.tif')
# (length1, y, x) = im2.shape
# im3 = io.imread('./data/AV_5_2.tif')
# (length2, y, x) = im3.shape

In [3]:
videoTS = np.ndarray.flatten(scio.loadmat('./data/camera_ts.mat')['cameraTime'])
wf_ts = np.ndarray.flatten(scio.loadmat('./data/wf_ts.mat')['cropped_wf'])

In [4]:
videoAns = []
for vidInd, vidTime in enumerate(wf_ts):
    windowT = vidTime
    if(vidTime > videoTS[0]) and (vidTime+3.85< videoTS[-1]):
        currIdx = find_nearest(videoTS, windowT)
        videoAns.append(currIdx)

In [9]:
vidIndexNP = im1[videoAns]
np.save("./data/whisk_df.npy", vidIndexNP)

In [ ]:
#either load above data set or if it has jus tbeen run then dont
#here

In [10]:
# whiskEncoder
whiskEncoder = load_model('whiskEncoder.h5')
input_img = Input(shape=(126, 136, 1)) 
w_l  = whiskEncoder.layers[0]
w_l1 = whiskEncoder.layers[1]
w_l2 = whiskEncoder.layers[2]
w_l3 = whiskEncoder.layers[3]
w_l4 = whiskEncoder.layers[4]
w_l5 = whiskEncoder.layers[5]

whisk_encoder = Model(input_img, w_l5(w_l4(w_l3(w_l2(w_l1(w_l(input_img)))))))

In [12]:
#get the size of the output of the last layer, create an array of that size 
#then for each image get the encoding
vidIndexNP = vidIndexNP.reshape(len(videoAns), 126,136,1)
encoded_images = whisk_encoder.predict(vidIndexNP)
np.save("./data/whisk_encoded_df.npy", encoded_images)